In [1]:
import os
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
driver = webdriver.Chrome('../chromedriver')
df = pd.read_csv('C:/rsource2/tour_data.csv')
df1 = df.loc[:,['tour_id','title']] 

In [3]:
df1.iloc[:50]

,tour_id,title
0,1,㈜코스테이
1,2,㈜피에프비파트너스 마린칸토
2,3,가나아트센터
3,4,가나안경원 (명동2호점)
4,5,가든파이브 (Garden5)
5,6,가락관광호텔
6,7,가락시장(가락농수산물 도매시장)
7,8,가람화랑
8,9,가로수길 홈스테이
9,10,가로수스테이


In [3]:
title = [] #관광지이름
comments = [] #리뷰
ratings = [] #별점
tour_ids = [] #tour_id
write_date = [] #리뷰작성시기

In [4]:
def main():

    global driver, load_wb, review_num

    driver.implicitly_wait(3)  # 렌더링 될때까지 기다린다 3초
    driver.get('https://map.kakao.com/')  # 주소 가져오기

    for i in range(0,20):
        #delapy
        sleep(2)
        print('####',i+1)
        search(df1['title'][i],i+1)

    driver.quit()
    print("finish")

In [5]:
# 카카오맵 첫화면에서 관광지명으로 검색한뒤 첫번째 결과로 crawilng 함수 호출하는 함수

def search(place,tour_id):
    global driver

    search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')  # 검색 창
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(1)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    try:
        soup = BeautifulSoup(html, 'html.parser')
        place= soup.select('.placelist > .PlaceItem')[0] # 검색된 장소 목록
        try:
            sleep(1)
            crawling(place,tour_id)

        except ElementNotInteractableException:
            print('not found')
        finally:
            search_area.clear()
    except:
        print('no search result')
        search_area.clear()

In [6]:
# 검색후 첫번째결과에 대해 상세정보를 새창으로 띄우는 함수

def crawling(place,tour_id):

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]'
        driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name, tour_id)

        # 첫 페이지
        extract_review(place_name,tour_id)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

In [7]:
#상세정보창에서 리뷰내용과 별점, 작성시기를 크롤링하는 함수

def extract_review(place_name,tour_id):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')

    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for i, review in enumerate(review_lists):
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            date = review.select('.time_write')[0].text #작성날짜
            val = ''
            
            title.append(place_name)
            tour_ids.append(tour_id)
            write_date.append(date)
            
            if len(comment) != 0:
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                else:
                    val = comment[0].text + '/0'

                comments.append(val)
                print(tour_id,val)

    #리뷰가 있고 여러페이지인경우..
    #해야됨
    else:
        print('no review in extract')
        ret = False

    return ret

In [8]:
if __name__ == "__main__":
    main()

#### 1
#### 코스테이 1
1 창문이 가려져있는데 나무로 만든 창문이라 도로소음을 자정넘게 들을 수 있습니다. 그리고 여러 후기가 있는데 경칩에 녹이 슬어있거나 문 틀이 일부 파손 및 노후화되어있으며 샤워 커튼대가 녹슨 경우도 있습니다. 세면대에 실리콘으로 벽면에 고정시키지 않은 상태도 있으며 거울 고정대부분에 녹슨 방도 있으니 방 잘 골라달라고 해야 합니다./2
#### 2
no search result
#### 3
#### 가나아트센터 3
3 경이로운 곳이었습니다/5
3 재밌게 구경잘함/4
3 최고예요/5
3 평창동은 그저 좋다../3
3 /0
#### 4
#### 가나안경원 명동2호점 4
4 명동이라 그런지 사람이 많긴많더라구요. 안경종류가 여러가지가 많고 가격이 저렴해서 자주 렌즈사러가는곳이에요./4
4 진짜 친절해요~~ 시력검사도 꼼꼼히 해주시고. 추천합니당/5
4 친절하고 잘 설명해 줘서 좋아요~~ 외국인 분들도 많던데ㅋ 번창하세요/5
#### 5
#### 가든파이브 5
5 정전기 때문에 다시오기 싫다. 무슨 피카츄가 사냐 손 닿는 곳 마다 빡빡!! /2
5 최고예요/5
5 아울렛 주변 빼고는 분위기가 무서움... 공실도 많아서.../3
5 아울렛과 지하 푸드코트가 좋아요. 특히 팝업 스토어/4
5 좋아요/4
#### 6
#### 가락관광호텔 6
6 롯데월드랑 롯데타워가기에 편했습니다. 방은 제법 넓은편이었고, 침구류도 깨끗한 편이네요./4
6 별 한개도 아깝다 진짜 무슨 배짱으로 저 가격을 받는건지.. 건물도 완전 낡았고 룸내에 중문은 닫기지도 않음 담배 냄새만 실컷 마시고 수건은 락스 냄새 진동을 하고(피부 트러블 남) 로비 프론트에 남자 직원 완전 불친절 네버 다시 안감!!!!!/1
6 802호실에 대실했는데 욕조에서 소변 찌린내나고 객실은 노후되어있고  침대시트는 교체했는지 모르겠네요...... 이름은 호텔인데 모텔보다 수준이하입니다. 정말 비추천!  안내데스크 남자직원은 예의 매너 없고 매우 불친절. /1
#### 7
#### 가락 

In [9]:
title

['코스테이',
 '가나아트센터',
 '가나아트센터',
 '가나아트센터',
 '가나아트센터',
 '가나아트센터',
 '가나안경원 명동2호점',
 '가나안경원 명동2호점',
 '가나안경원 명동2호점',
 '가든파이브',
 '가든파이브',
 '가든파이브',
 '가든파이브',
 '가든파이브',
 '가락관광호텔',
 '가락관광호텔',
 '가락관광호텔',
 '가마목',
 '가마목',
 '가마목',
 '가마목',
 '가마목',
 '가문애족발',
 '가시리 본점',
 '가시리 본점',
 '가시리 본점',
 '가시리 본점',
 '가시리 본점',
 '가회민화박물관',
 '갈산근린공원',
 '갈산근린공원',
 '감꽃당',
 '감꽃당',
 '감꽃당',
 '감꽃당',
 '감꽃당']

In [10]:
for i in range(len(comments)):
    ratings.append(str(comments[i])[-1])
    comments[i] = comments[i][:-2]
    write_date[i] = write_date[i][:-1]

In [11]:
comments

['창문이 가려져있는데 나무로 만든 창문이라 도로소음을 자정넘게 들을 수 있습니다. 그리고 여러 후기가 있는데 경칩에 녹이 슬어있거나 문 틀이 일부 파손 및 노후화되어있으며 샤워 커튼대가 녹슨 경우도 있습니다. 세면대에 실리콘으로 벽면에 고정시키지 않은 상태도 있으며 거울 고정대부분에 녹슨 방도 있으니 방 잘 골라달라고 해야 합니다.',
 '경이로운 곳이었습니다',
 '재밌게 구경잘함',
 '최고예요',
 '평창동은 그저 좋다..',
 '',
 '명동이라 그런지 사람이 많긴많더라구요. 안경종류가 여러가지가 많고 가격이 저렴해서 자주 렌즈사러가는곳이에요.',
 '진짜 친절해요~~ 시력검사도 꼼꼼히 해주시고. 추천합니당',
 '친절하고 잘 설명해 줘서 좋아요~~ 외국인 분들도 많던데ㅋ 번창하세요',
 '정전기 때문에 다시오기 싫다. 무슨 피카츄가 사냐 손 닿는 곳 마다 빡빡!! ',
 '최고예요',
 '아울렛 주변 빼고는 분위기가 무서움... 공실도 많아서...',
 '아울렛과 지하 푸드코트가 좋아요. 특히 팝업 스토어',
 '좋아요',
 '롯데월드랑 롯데타워가기에 편했습니다. 방은 제법 넓은편이었고, 침구류도 깨끗한 편이네요.',
 '별 한개도 아깝다 진짜 무슨 배짱으로 저 가격을 받는건지.. 건물도 완전 낡았고 룸내에 중문은 닫기지도 않음 담배 냄새만 실컷 마시고 수건은 락스 냄새 진동을 하고(피부 트러블 남) 로비 프론트에 남자 직원 완전 불친절 네버 다시 안감!!!!!',
 '802호실에 대실했는데 욕조에서 소변 찌린내나고 객실은 노후되어있고  침대시트는 교체했는지 모르겠네요...... 이름은 호텔인데 모텔보다 수준이하입니다. 정말 비추천!  안내데스크 남자직원은 예의 매너 없고 매우 불친절. ',
 '최고예요',
 '점심에 쌈밥 짱맛있어용 볶음밥도 꿀맛',
 '삼겹살 너무 맛있고 직접 고기도 구워주시고 정말 친절한 식당입니다!',
 '주문하면 화내는집. 고기주문할려고 벨 누를때마다 아주머니가 한숨쉬면서 짜증내면서 오심 ㅋㅋㅋ 그런식으로 할꺼면 장사하지마세

In [12]:
temp1 = pd.DataFrame({'tour_id':tour_ids,'title':title,'comments':comments,'ratings':ratings,'date':write_date})

In [13]:
temp1

,tour_id,title,comments,ratings,date
0,1,코스테이,창문이 가려져있는데 나무로 만든 창문이라 도로소음을 자정넘게 들을 수 있습니다. 그...,2,2019.03.16
1,3,가나아트센터,경이로운 곳이었습니다,5,2020.02.16
2,3,가나아트센터,재밌게 구경잘함,4,2020.02.11
3,3,가나아트센터,최고예요,5,2018.02.16
4,3,가나아트센터,평창동은 그저 좋다..,3,2016.04.23
5,3,가나아트센터,,0,2014.06.30
6,4,가나안경원 명동2호점,명동이라 그런지 사람이 많긴많더라구요. 안경종류가 여러가지가 많고 가격이 저렴해서 ...,4,2015.09.10
7,4,가나안경원 명동2호점,진짜 친절해요~~ 시력검사도 꼼꼼히 해주시고. 추천합니당,5,2013.12.18
8,4,가나안경원 명동2호점,친절하고 잘 설명해 줘서 좋아요~~ 외국인 분들도 많던데ㅋ 번창하세요,5,2013.11.04
9,5,가든파이브,정전기 때문에 다시오기 싫다. 무슨 피카츄가 사냐 손 닿는 곳 마다 빡빡!!,2,2020.10.20
